<a href="https://colab.research.google.com/github/illliyas/Machine-Learning/blob/master/tweets/Live_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bertopic
!pip install joblib==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
#Lets import important libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from bertopic import BERTopic

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [5]:
df = pd.read_csv('ek_tweets4.csv')

In [6]:
sw = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [7]:
def cleaning_tweets(tweet):
    temp = tweet.lower()
    #Removing hashtags and mentions
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    #Removing links
    temp = re.sub(r"http\S+", "", temp)
    temp = re.sub(r"www.\S+", "", temp)
    #Removing punctuations
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #Filtering non-alphanumeric characters
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = re.sub("([a-z]*[0-9]+[a-z0-9]* )+"," ", temp)
    #remove 2 letter words
    temp = re.sub(r"\b[a-z]{2}\b"," ", temp)
    return temp

def remove_stopwords(tweet):
    words = word_tokenize(tweet)
    words = [word for word in words if word not in sw]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words) 
def number_of_words(tweet):
  return len(word_tokenize(tweet))     

In [8]:
df['tweet_clean'] = df.apply(lambda row: cleaning_tweets(row['tweet']),axis=1)
df['tweet_clean2'] = df.apply(lambda row: remove_stopwords(row['tweet_clean']),axis=1)
df['number_of_words'] = df.apply(lambda row: number_of_words(row['tweet_clean2']),axis=1)

In [9]:
df2 = df[df['number_of_words']>2]
df2=df2.reset_index()

In [22]:
list(df2.tweet)[12:20]

['@EmiratesSupport @emirates can you advise why we are now being quote £900+ for a room which, when we booked were told was included? https://t.co/tbMzz3bONr',
 '@EmiratesSupport any particular reason why your website is so shit? Online forms telling me to only use “valid characters” like I’m a cartoonist. Your site is shit and your customer service is abysmal',
 '@EmiratesSupport my elderly mother is suffering in Amsterdam airport and you took only first and business class people to the hotels and left everyone.. SHAME ON YOU',
 '@EmiratesSupport @emirates is this how the "best airline" in the world treats their passengers? You will be sued for mishandling all this. I read your terms and conditions',
 "@EmiratesSupport Yeah pointless. Saying you can't book a hotel stay nor upgrade... this is basic EU regulation EC 261.... a flight cancellation of over 24 hrs and more than 3,500km",
 '@EmiratesSupport, hi I need urgent help  as need to change  my tickets I was trying to call the helpli

In [23]:
list(df2.tweet_clean2)[12:20]

['advise quote room booked told included',
 'particular reason website shit online form telling use valid character like cartoonist site shit customer service abysmal',
 'elderly mother suffering amsterdam airport took first business class people hotel left everyone shame',
 'best airline world treat passenger sued mishandling read term condition',
 'yeah pointless saying book hotel stay upgrade basic regulation flight cancellation hr 500km',
 'need urgent help need change ticket trying call helpline since morning could get way contact',
 'team emirate changed trip start date stating operation reason due whole schedule impacted request change return date well without charge fault',
 'thanks much however last updated july thing moved fair bit since']

In [11]:
df2['created_at'] = pd.to_datetime(df2['created_at'])
tweets = df2.tweet_clean2.to_list()
timestamps  = df2.created_at.to_list()

In [23]:
model1 = BERTopic(verbose=True,embedding_model='all-MiniLM-L6-v2', min_topic_size= 6)
topics, _ = model1.fit_transform(tweets)

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

2022-11-05 20:03:04,187 - BERTopic - Transformed documents to Embeddings
2022-11-05 20:03:08,480 - BERTopic - Reduced dimensionality
2022-11-05 20:03:08,530 - BERTopic - Clustered reduced embeddings


In [24]:
model1.get_topic_info()

,Topic,Count,Name
0,-1,110,-1_flight_response_get_time
1,0,90,0_booking_change_help_flight
2,1,73,1_luggage_baggage_bag_emirate
3,2,41,2_complaint_customer_reply_email
4,3,38,3_please_sent_thank_reply
5,4,29,4_emirate_support_time_date
6,5,21,5_meal_nut_food_infant
7,6,20,6_vaccinated_test_vaccination_pcr
8,7,17,7_refund_rebook_cancelled_money
9,8,16,8_invoice_much_form_resolving


In [27]:
df2['tweet_topics'] = topics
model1.topic_labels_

{-1: '-1_flight_response_get_time',
 0: '0_booking_change_help_flight',
 1: '1_luggage_baggage_bag_emirate',
 2: '2_complaint_customer_reply_email',
 3: '3_please_sent_thank_reply',
 4: '4_emirate_support_time_date',
 5: '5_meal_nut_food_infant',
 6: '6_vaccinated_test_vaccination_pcr',
 7: '7_refund_rebook_cancelled_money',
 8: '8_invoice_much_form_resolving',
 9: '9_staff_delhi_airline_made',
 10: '10_lounge_access_business_flying',
 11: '11_visa_transit_dubai_admins',
 12: '12_saturday_system_decided_shoul',
 13: '13_email_pathetic_whenever_equally',
 14: '14_knee_twin_family_dxb',
 15: '15_dublin_five_exhausted_flight',
 16: '16_selling_money_robbed_scammed',
 17: '17_complaint_nothing_emirate_made'}

In [29]:
model1.visualize_topics()

In [32]:
model1.visualize_barchart()

In [62]:
topics_over_time = model1.topics_over_time(tweets, timestamps)

model1.visualize_topics_over_time(topics_over_time, topics=[0, 1, 2, 3, 9])

530it [00:05, 98.39it/s] 


In [36]:
type(model1.topic_labels_)

dict

In [39]:
topics_df = pd.DataFrame(model1.topic_labels_.items(), columns=['topic_id', 'topic_name'])

In [41]:
df2 = df2.merge(topics_df,how='left',left_on='tweet_topics',right_on='topic_id')

In [49]:
df2['like_count'] = df2['like_count'].astype(int)

In [52]:
df2[['like_count','tweet','topic_name']].sort_values('like_count',ascending=False)

,like_count,tweet,topic_name
162,290,@EmiratesSupport just tried to check in for ou...,12_saturday_system_decided_shoul
270,141,@EmiratesSupport It doesn’t help. And I’m not ...,1_luggage_baggage_bag_emirate
169,10,@EmiratesSupport Thank you so much for resolvi...,8_invoice_much_form_resolving
72,7,@EmiratesSupport I'm not the type to complain ...,0_booking_change_help_flight
225,3,@EmiratesSupport hello we’re looking to fly to...,5_meal_nut_food_infant
...,...,...,...
180,0,@EmiratesSupport I get that now calling is not...,-1_flight_response_get_time
179,0,@EmiratesSupport since 2019 Feb till today the...,4_emirate_support_time_date
178,0,@EmiratesSupport @EmiratesSupport @emirates th...,0_booking_change_help_flight
177,0,@EmiratesSupport quick question is the second ...,0_booking_change_help_flight


In [57]:
topics_df

,topic_id,topic_name
0,-1,-1_flight_response_get_time
1,0,0_booking_change_help_flight
2,1,1_luggage_baggage_bag_emirate
3,2,2_complaint_customer_reply_email
4,3,3_please_sent_thank_reply
5,4,4_emirate_support_time_date
6,5,5_meal_nut_food_infant
7,6,6_vaccinated_test_vaccination_pcr
8,7,7_refund_rebook_cancelled_money
9,8,8_invoice_much_form_resolving


In [63]:
list(df2[df2['topic_name']=='1_luggage_baggage_bag_emirate']['tweet'])

['@EmiratesSupport Hi, we are flying economy from LGW to DXB on 15th October. What are our luggage allowances? Your website says 23kg hold + 7kg in the cabin. My ticket from you just says 30kg with no reference to either hold or cabin and my ticket from my TA says 20kg. Thanks',
 '@EmiratesSupport I pay over £5K for business and the baggage is supposed to be a priority, and yet I’m still here and no bags 45 minutes later. Booked my driver for 8.45 and it’s almost 10pm! https://t.co/CwcoWlTTkb',
 '@EmiratesSupport are we able to add a pram in the hold for our 1yr old as part of our luggage allowance &amp; if so, can we hold on to it until boarding at the gate? London -&gt; Dubai &amp; vice versa',
 '@EmiratesSupport   Hi I am a frequent flyer of emirates airlines I traveled from emirate’s airlines on 18th of May to JFK USA on arrival I found my one of my checkin baggage broken.I complain to your emirates counter at JFK .They give me complain no JFKD550185.Till date not solve',
 '@Emirat

In [55]:
df2.iloc[162]['tweet']

'@EmiratesSupport just tried to check in for our flight Saturday and your system has decided that my 4 and 6 year old should be sitting on opposite ends of the plane away from my wife and I 😵\u200d💫'

In [56]:
df2.iloc[270]['tweet']

'@EmiratesSupport It doesn’t help. And I’m not happy being ripped off. Now what can you do? 🤣'